<a href="https://colab.research.google.com/github/MrKyZs/Trabalho-4-semestre/blob/main/TDE_4_Semestre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
#import plotly.express as px
#pd.__version__

In [ ]:
path = "TabelaGasolinas.csv"
r_df = pd.read_csv(path, sep=';')

pathkek = "TabelaIPCA.csv"
kek_df = pd.read_csv(pathkek, sep=';')

In [ ]:
#São escolhidas as colunas que serão passadas para o novo dataframe
columns = r_df[['MES', 'ESTADO', 'PRODUTO', 'PRECO MEDIO REVENDA']]
df = columns.copy()

df['PRECO MEDIO REVENDA'] = df['PRECO MEDIO REVENDA'].str.replace(',','.')
todos_estados = np.unique(df['ESTADO'])

estados = ['SAO PAULO', 'RIO DE JANEIRO', 'ESPIRITO SANTO', 'MINAS GERAIS']
#estados = todos_estados
#estados = ['MARANHAO', 'PIAUI', 'CEARA', 'RIO GRANDE DO NORTE', 'PARAIBA', 'PERNAMBUCO', 'ALAGOAS', 'SERGIPE', 'BAHIA']

# Correlação de Pearson

In [ ]:
#Pega as operações mean e std utilizando uma função que recebe como atributos uma coluna e o dataframe. 
#É feito o calculo da media e do desvio usando os metodos .mean e .std, e os resultados são inserido em novas colunas 
#Que são criadas usando o nome da coluna passada como atributo da função mais o sufixo OP, onde é armazenado o valor da coluna original
#menos a media, dividido pelo desvio.
def operacao(col, df_completo):
  mean = df_completo[col].mean()
  std = df_completo[col].std()
  df_completo[col+' OP'] = (df_completo[col]-mean)/std
  return mean, std, df_completo

In [ ]:
#Gera a primeira coluna do dataframe final com o primeiro estado da array

#Pega apenas a as colunas que possuem PRODUTO = Gasolina Comum do primeiro estado selecionado, juntamente com o preço medio de revenda e passa para float usando
#O metodo .astype() que recebe como atributo o tipo que será convertido para. 
df_gas = df[(df['PRODUTO'] == 'GASOLINA COMUM')][df['ESTADO'] == estados[0]]
df_gas['PRECO MEDIO REVENDA'] = df_gas['PRECO MEDIO REVENDA'].astype(float)
df_ip = kek_df.copy()

#Junta os meses e anos para fica no formato jan/20.
df_ip['MES'] = kek_df["MES"].str.lower() + '/'+ kek_df['ANO'].astype(str).str[-2:]

df_ip = df_ip.drop('ANO', axis=1)

#Junta os datasets da gasolina e o do ipca 
df_all = pd.merge(df_ip, df_gas, on='MES')

df_all['UM MES'] = df_all['UM MES'].str.replace(',','.').astype(float)

df_all = df_all.drop(['NUMERO INDICE','TRES MESES','SEIS MESES', 'UM ANO', 'DOZE MESES'], axis=1)

#Pega a variação percentual com o metodo pct_change() e faz * 100 para que os valores fiquem melhores para serem trabalhados.
variacao = df_all['PRECO MEDIO REVENDA'].pct_change()*100

#Cria uma coluna no dataframe para armazenar os valores das variações percentuais
df_all['PRECO MEDIO REVENDA VAR'] = variacao

icpa = df_all['UM MES']

#É feito o calculo da media e do desvio usando os metodos .mean e .std, e os resultados são inserido em novas colunas 
operacao('PRECO MEDIO REVENDA VAR', df_all)
operacao('UM MES', df_all)

#Cria a uma Series chamada pearson que vai armazenar a multiplicação entre a coluna PRECO MEDIO REVENDA OP
df_all['Pearson'] = df_all['PRECO MEDIO REVENDA VAR OP'] * df_all['UM MES OP']

#Cria uma Series chamada Pearson que será armazena os valores da coluna Pearson
pearson = df_all['Pearson']

df_final = df_all.drop(['PRECO MEDIO REVENDA VAR','UM MES OP', 'PRECO MEDIO REVENDA VAR OP', 'ESTADO', 'PRODUTO', 'PRECO MEDIO REVENDA', 'UM MES'], axis=1)

#Calcula a média dos indices de pearson gerados e coloca no df_final. Além de renomear as colunas utilizando o metodo
#Rename, que recebe como padrão o nome atual da coluna, seguido de dois pontos, e o que será renomeado para.
df_final = df_final.rename(columns={"Pearson": "ESTADO: %s | INDICE PEARSON %.4f" % (estados[0], pearson.mean())})

#Como no Series Pearson está o indice de correlação mensal, precisamos calcular a média de todas as linhas da Series
Sudeste = pearson.mean()

#É realizado um for para iterar em todos os estados presentes na array "estados" a partir do primeiro item
#Que é feito em cima.
for estado in estados[1:]:
  second_df = df[(df['PRODUTO'] == 'GASOLINA COMUM')][df['ESTADO'] == estado]
  second_df['PRECO MEDIO REVENDA'] = second_df['PRECO MEDIO REVENDA'].astype(float)

  ipca_df2 = kek_df.copy()
  ipca_df2['MES'] = kek_df["MES"].str.lower() + '/'+ kek_df['ANO'].astype(str).str[-2:]
  df_usable = pd.merge(second_df,ipca_df2, on='MES')
  df_usable['UM MES'] = df_usable['UM MES'].str.replace(',','.').astype(float)
  df_usable = df_usable.drop(['NUMERO INDICE','TRES MESES','SEIS MESES', 'UM ANO', 'DOZE MESES', 'ANO'], axis=1)

  variacao = df_usable['PRECO MEDIO REVENDA'].pct_change()*100
  df_usable['PRECO MEDIO REVENDA VAR'] = variacao
  icpa = df_usable['UM MES']

  operacao('PRECO MEDIO REVENDA VAR', df_usable)
  operacao('UM MES', df_usable)
  df_usable['Pearson'] = df_usable['PRECO MEDIO REVENDA VAR OP'] * df_usable['UM MES OP']
  pearson = df_usable['Pearson']
  df_usable = df_usable.drop(['PRECO MEDIO REVENDA VAR','UM MES OP', 'PRECO MEDIO REVENDA VAR OP', 'ESTADO', 'PRODUTO', 'PRECO MEDIO REVENDA', 'UM MES'], axis=1)
  
  df_usable = df_usable.rename(columns={"Pearson": "ESTADO: %s | INDICE PEARSON %.4f" % (estado, pearson.mean())})
  df_final = df_final.merge(df_usable, on='MES')

  Sudeste = Sudeste + pearson.mean()

ax = df_final.plot(title ="Correlação Gasolina com IPCA", x = 'MES', figsize=(20,10))
ax.set_ylabel("IPCA")
#print(Sudeste/len(estados))